In [31]:
import joblib
from pathlib import Path
import time
import vaex
import vaex.ml
from datetime import datetime
import dask
import dask.array as da
import numpy as np
from dask.distributed import Client

dir = Path().cwd()
csv = os.path.join(dir, 'output', 'gtfsr_combined_csv.csv')
scats = os.path.join(dir, 'output', 'scats_model.json')


In [32]:
start = time.time()
scats_model = joblib.load(scats)

print(time.time()-start)

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.29 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.30 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.33 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.35 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.37 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARN

KeyboardInterrupt: 

In [5]:
if not os.path.exists(csv+'.hdf5'):
    df = vaex.from_csv(csv, convert=True, copy_index=False, chunk_size=1000000)
    df.export(csv, shuffle=True)
df = vaex.open(csv+'.hdf5')
df.head()

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,lon,lat
0,11777.2.60-13-b12-1.22.I,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
1,11756.2.60-13-b12-1.22.I,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
2,11370.2.60-13-b12-1.23.I,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.930786,-0.112593
3,11379.2.60-13-b12-1.19.O,2.02101e+07,18:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
4,11394.2.60-13-b12-1.19.O,2.02101e+07,18:15:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
5,8595.2.60-4-b12-1.5.O,2.02101e+07,18:30:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
6,8566.2.60-4-b12-1.5.O,2.02101e+07,18:45:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
7,11401.2.60-13-b12-1.19.O,2.02101e+07,18:45:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
8,11662.2.60-27B-b12-1.97.I,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583
9,8924.2.60-4-b12-1.5.O,2.02101e+07,19:00:00,1,0,0,2021-01-09 19:28:02,0.932315,-0.109583


In [6]:
len(df)

3100815

In [7]:
def apply_dow(t):
    return datetime.strptime(str(t), '%Y%m%d').weekday()

def apply_day(t):
    return datetime.strptime(str(t), '%Y%m%d').day

def apply_month(t):
    return datetime.strptime(str(t), '%Y%m%d').month

def apply_hour(t):
    return datetime.strptime(t, '%H:%M:%S').hour

df['dow'] = df['start_date'].apply(apply_dow)
df['month'] = df['start_date'].apply(apply_month)
df['day'] = df['start_date'].apply(apply_day)
df['hour'] = df['start_time'].apply(apply_hour)
df


#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,lon,lat,dow,month,day,hour
0,11777.2.60-13-b12-1.22.I,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,18
1,11756.2.60-13-b12-1.22.I,20210109,18:15:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,18
2,11370.2.60-13-b12-1.23.I,20210109,19:00:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,19
3,11379.2.60-13-b12-1.19.O,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,0.9323151396966098,-0.1095830133123873,5,1,9,18
4,11394.2.60-13-b12-1.19.O,20210109,18:15:00,1,0,0,2021-01-09 19:28:02,0.9323151396966098,-0.1095830133123873,5,1,9,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"3,100,810",4019.10447.2-76-gad-1.67.I,20210131,23:25:00,31,-120,-120,2021-01-31 23:57:43,0.9300938156686116,-0.1111662507782495,6,1,31,23
"3,100,811",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,1,0,0,2021-01-31 23:57:43,0.9318992722701834,-0.1115625707141637,6,1,31,23
"3,100,812",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,6,60,60,2021-01-31 23:57:43,0.932067467446198,-0.1119520695404478,6,1,31,23
"3,100,813",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,1,0,0,2021-01-31 23:58:50,0.9318992722701834,-0.1115625707141637,6,1,31,23


In [8]:
df.start_time.dtype

str

In [9]:
df.start_time = df.start_time.fillmissing('')

In [10]:
df.start_time.isna().sum()

array(0, dtype=int64)

In [11]:
# convert to dask

dask_df = df[['lon', 'lat']].to_dask_array()
dask_df

dask.array<vaex-df-5f7469f4-7065-11eb-a146, shape=(3100815, 2), dtype=float64, chunksize=(3100815, 2), chunktype=numpy.ndarray>

In [12]:
def extract_scats_from_gtfsr(df):
    new_df = df
    pca_coord = vaex.ml.PCA(features=['lat', 'lon'], n_components=2, prefix='pca')
    new_df = pca_coord.fit_transform(new_df)

    cycl_transform_hour = vaex.ml.CycleTransformer(features=['hour'], n=24)
    new_df = cycl_transform_hour.fit_transform(new_df)

    cycl_transform_dow = vaex.ml.CycleTransformer(features=['dow'], n=7)
    new_df = cycl_transform_dow.fit_transform(new_df)

    feats = new_df.get_column_names(regex='pca') + \
        new_df.get_column_names(regex='.*_x') + \
        new_df.get_column_names(regex='.*_y')

    return new_df

train_df = extract_scats_from_gtfsr(df)
train_df = scats_model.transform(train_df)
train_df


#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,lon,lat,dow,month,day,hour,pca0,pca1,hour_x,hour_y,dow_x,dow_y,p_avg_vol
0,11777.2.60-13-b12-1.22.I,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,18,0.0031128146182939904,0.0016488832444477768,-1.8369701987210297e-16,-1.0,-0.2225209339563146,-0.9749279121818236,2.145520673463261
1,11756.2.60-13-b12-1.22.I,20210109,18:15:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,18,0.0031128146182939904,0.0016488832444477768,-1.8369701987210297e-16,-1.0,-0.2225209339563146,-0.9749279121818236,2.145520673463261
2,11370.2.60-13-b12-1.23.I,20210109,19:00:00,1,0,0,2021-01-09 19:28:02,0.9307856422222316,-0.112592634936091,5,1,9,19,0.0031128146182939904,0.0016488832444477768,0.2588190451025203,-0.9659258262890684,-0.2225209339563146,-0.9749279121818236,0.6852928250850904
3,11379.2.60-13-b12-1.19.O,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,0.9323151396966098,-0.1095830133123873,5,1,9,18,0.0011608645568870167,-0.001105581458496806,-1.8369701987210297e-16,-1.0,-0.2225209339563146,-0.9749279121818236,61.03036081311585
4,11394.2.60-13-b12-1.19.O,20210109,18:15:00,1,0,0,2021-01-09 19:28:02,0.9323151396966098,-0.1095830133123873,5,1,9,18,0.0011608645568870167,-0.001105581458496806,-1.8369701987210297e-16,-1.0,-0.2225209339563146,-0.9749279121818236,61.03036081311585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"3,100,810",4019.10447.2-76-gad-1.67.I,20210131,23:25:00,31,-120,-120,2021-01-31 23:57:43,0.9300938156686116,-0.1111662507782495,6,1,31,23,0.001528345988793678,0.0015973574434904186,0.9659258262890681,-0.25881904510252157,0.6234898018587334,-0.7818314824680299,13.016276191884726
"3,100,811",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,1,0,0,2021-01-31 23:57:43,0.9318992722701834,-0.1115625707141637,6,1,31,23,0.0027194086330715244,0.000183811261305979,0.9659258262890681,-0.25881904510252157,0.6234898018587334,-0.7818314824680299,56.534833885976944
"3,100,812",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,6,60,60,2021-01-31 23:57:43,0.932067467446198,-0.1119520695404478,6,1,31,23,0.00314243179910769,0.00021622042525668082,0.9659258262890681,-0.25881904510252157,0.6234898018587334,-0.7818314824680299,56.534833885976944
"3,100,813",7791.10447.2-270-gad-1.235.I,20210131,23:40:00,1,0,0,2021-01-31 23:58:50,0.9318992722701834,-0.1115625707141637,6,1,31,23,0.0027194086330715244,0.000183811261305979,0.9659258262890681,-0.25881904510252157,0.6234898018587334,-0.7818314824680299,56.534833885976944


In [13]:
train_df['p_avg_vol']

Expression = p_avg_vol
Length: 3,100,815 dtype: float64 (column)
-----------------------------------------
      0   2.14552
      1   2.14552
      2  0.685293
      3   61.0304
      4   61.0304
       ...       
3100810   13.0163
3100811   56.5348
3100812   56.5348
3100813   56.5348
3100814   56.5348

In [24]:
dask_p = train_df[['p_avg_vol']].to_dask_array()
dask_p

dask.array<vaex-df-f233d50a-7065-11eb-94e3, shape=(3100815, 1), dtype=float64, chunksize=(3100815, 1), chunktype=numpy.ndarray>

In [27]:
r = np.mean(dask_p)
r

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [30]:
r_computed = r.compute()
r_computed

distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.17 GB -- Worker memory limit: 4.00 GB
distributed.worker - WARNING - Memory use is high but worker has no data to store to disk.  Perhaps some other process is leaking memory?  Process memory: 3.17 GB -- Worker memory limit: 4.00 GB
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x00000215A56AC548>>, <Task finished coro=<Worker.memory_monitor() done, defined at C:\Users\vmedv\Anaconda3\lib\site-packages\distributed\worker.py:2518> exception=TypeError('Could not serialize object of type DataFrameArrays.', '#          p_avg_vol\n0          2.145520673463261\n1          2.145520673463261\n2          0.6852928250850904\n3          61.03036081311585\n4          61.03036081311585\n...        ...\n3,100,810  13.016276191884726\n

KeyboardInterrupt: 